# 🧠 GUÍA PRÁCTICA 1 — Análisis de Datos con Python

**Alumno:** _Maura Calle_  
**Asignatura:** Inteligencia Artificial — Práctica 1  
**Objetivo:** Realizar un Análisis Exploratorio de Datos (EDA) aplicado a compras públicas en Ecuador, incluyendo carga, limpieza, visualización y análisis por años.


## 🧩 1. Importación de librerías necesarias  
**Objetivo:** Importar las bibliotecas esenciales que se usarán para el análisis de datos, limpieza y visualización.  

**Descripción:**  
- **pandas:** manejo de datos en DataFrame.  
- **numpy:** cálculos numéricos.  
- **requests:** conexión a APIs.  
- **plotly:** visualizaciones interactivas.  
- **datetime:** manejo de fechas.


In [ ]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
import io

## 📂 2. Carga de datos desde API o CSV  
**Objetivo:** Obtener los datos desde la API pública o un CSV local.  
**Descripción:** Garantiza que el flujo funcione aunque la API falle, usando un CSV o un dataset demo.


In [ ]:
API_URL = "https://datosabiertos.compraspublicas.gob.ec/PLATAFORMA/api/search_ocds"
LOCAL_CSV = "compras_publicas_ecuador.csv"

def load_from_api(params=None, timeout=15):
    try:
        resp = requests.get(API_URL, params=params or {}, timeout=timeout)
        resp.raise_for_status()
        data = resp.json()
        if isinstance(data, dict) and "data" in data:
            df = pd.json_normalize(data["data"])
        else:
            df = pd.json_normalize(data)
        print("✅ Datos cargados correctamente desde la API.")
        return df
    except Exception as e:
        print(f"⚠️ Error al cargar desde API: {e}")
        return None

def load_from_csv(path):
    try:
        df = pd.read_csv(path)
        print("✅ Datos cargados desde CSV local.")
        return df
    except Exception as e:
        print(f"⚠️ Error al cargar CSV: {e}")
        return None

def load_data(api_params=None, csv_path=LOCAL_CSV):
    df = load_from_api(api_params)
    if df is None:
        df = load_from_csv(csv_path)
    if df is None:
        print("⚙️ Generando dataset demo.")
        rng = pd.date_range("2021-01-01", "2023-12-31", freq="W")
        n = len(rng)
        df = pd.DataFrame({
            "date": rng,
            "province": np.random.choice(["Azuay","Pichincha","Guayas","Manabí"], size=n),
            "internal_type": np.random.choice(["Obra","Suministro","Servicios"], size=n),
            "contracts": np.random.poisson(3, size=n),
            "total": np.abs(np.random.normal(50000, 20000, size=n)).round(2)
        })
    return df

df = load_data()


## 🧼 3. Limpieza y estandarización de datos  
**Objetivo:** Preparar el dataset eliminando duplicados, nulos y renombrando columnas.


In [ ]:
def clean_dataframe(df):
    df = df.copy()
    rename_map = {}
    for col in df.columns:
        low = col.lower().strip()
        if any(x in low for x in ["provincia", "province"]):
            rename_map[col] = "province"
        elif any(x in low for x in ["tipo", "contratacion", "internal_type", "type"]):
            rename_map[col] = "internal_type"
        elif any(x in low for x in ["monto", "total", "valor", "amount", "price"]):
            rename_map[col] = "total"
        elif any(x in low for x in ["contrato", "contracts", "nro_contratos"]):
            rename_map[col] = "contracts"
        elif any(x in low for x in ["fecha", "date"]):
            rename_map[col] = "date"
        elif "year" in low or "año" in low:
            rename_map[col] = "year"
        elif "month" in low or "mes" in low:
            rename_map[col] = "month"
    if rename_map:
        df = df.rename(columns=rename_map)
    if "total" not in df.columns:
        df["total"] = 0.0
    if "contracts" not in df.columns:
        df["contracts"] = 0
    if "date" not in df.columns:
        df["date"] = pd.NaT
    df["total"] = pd.to_numeric(df["total"], errors="coerce")
    df["contracts"] = pd.to_numeric(df["contracts"], errors="coerce").fillna(0).astype(int)
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    if df["date"].notna().any():
        df["year"] = df["date"].dt.year
        df["month"] = df["date"].dt.month
    else:
        if "year" not in df.columns: df["year"] = np.nan
        if "month" not in df.columns: df["month"] = np.nan
    df = df.drop_duplicates()
    df = df.dropna(subset=["total"], how="all")
    for c in ["province", "internal_type"]:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip().str.title().replace({"Nan": "Desconocido", "None": "Desconocido"})
    return df

df = clean_dataframe(df)
df.info()


## 📊 4. Verificación inicial y descripción de datos  
**Objetivo:** Explorar la estructura y tipos de datos.


In [ ]:
print(df.shape)
df.head()
df.describe()


## 📊 5. KPIs básicos  
**Objetivo:** Calcular indicadores principales como registros, montos totales, promedio, máximo y mínimo.


In [ ]:
total_registros = len(df)
monto_total = df["total"].sum()
promedio = df["total"].mean()
maximo = df["total"].max()
minimo = df["total"].min()

print("📈 KPIs Principales")
print("----------------------")
print(f"Total de registros: {total_registros}")
print(f"Monto total: ${monto_total:,.2f}")
print(f"Promedio por registro: ${promedio:,.2f}")
print(f"Máximo: ${maximo:,.2f}")
print(f"Mínimo: ${minimo:,.2f}")


## 📊 6. Análisis por categorías  
**Objetivo:** Ver distribución por tipo de contratación y provincia.


In [ ]:
print("Contrataciones por tipo:")
print(df["internal_type"].value_counts())

print("\nMontos totales por provincia:")
print(df.groupby("province")["total"].sum().sort_values(ascending=False).head(10))


## 📈 7. Visualización: Montos por tipo de contratación


In [ ]:
fig1 = px.bar(
    df.groupby("internal_type")["total"].sum().reset_index().sort_values("total", ascending=False),
    x="internal_type",
    y="total",
    title="Total de montos por tipo de contratación",
    labels={"internal_type":"Tipo de Contratación", "total":"Monto Total (USD)"}
)
fig1.show()


## 📆 8. Evolución mensual de montos


In [ ]:
if df["date"].notna().sum() > 0:
    monthly = df.set_index("date").resample("M")["total"].sum().reset_index()
    fig2 = px.line(monthly, x="date", y="total", title="Evolución mensual de montos totales")
    fig2.show()


## 🔗 9. Relación contratos vs montos


In [ ]:
fig3 = px.scatter(df, x="contracts", y="total", color="internal_type",
                  title="Relación entre contratos y montos",
                  labels={"contracts":"Cantidad de Contratos","total":"Monto Total (USD)"})
fig3.show()


## 🗓️ 10. Análisis por años


In [ ]:
kpis_year = df.groupby("year").agg(
    registros=("total","count"),
    monto_total=("total","sum"),
    promedio=("total","mean")
).reset_index()
kpis_year


## 🌡️ 11. Heatmap Año × Mes


In [ ]:
if df["date"].notna().sum() > 0:
    heat = df.copy()
    heat["year"] = heat["date"].dt.year
    heat["month"] = heat["date"].dt.month
    pivot_h = heat.pivot_table(index="year", columns="month", values="total", aggfunc="sum", fill_value=0)
    fig4 = px.imshow(pivot_h, aspect="auto", title="Heatmap Año × Mes", labels={"x":"Mes","y":"Año","color":"Monto"})
    fig4.show()


## 📉 12. Correlación contratos vs montos


In [ ]:
corr = df[["total","contracts"]].corr()
corr


## 💾 13. Exportación de resultados


In [ ]:
df.to_csv("compras_publicas_procesadas.csv", index=False)
print("✅ CSV exportado con éxito.")


## 🧭 14. Conclusiones finales  
**Objetivo:** Sintetizar hallazgos: tendencias por tipo de contrato, provincias con mayores montos, variaciones por año y mes.
